In [1]:
#Import Required Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [2]:
#Load Cleaned Data
df = pd.read_csv('../data/processed_data/cleaned_data.csv')
print(df.columns)


Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday_x', 'Temperature',
       'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4',
       'MarkDown5', 'CPI', 'Unemployment', 'IsHoliday_y', 'Type', 'Size'],
      dtype='object')


In [4]:
# Load the cleaned data
df = pd.read_csv('../data/processed_data/cleaned_data.csv')

# Drop non-relevant or duplicate columns
df = df.drop(columns=['Date'])

# One-hot encode categorical columns if not already encoded
df = pd.get_dummies(df, columns=['Type'], drop_first=True)

# Features and target
X = df.drop('Weekly_Sales', axis=1)
y = df['Weekly_Sales']

# Normalize features
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [5]:
print(X.dtypes)
print(X.head())


Store             int64
Dept              int64
IsHoliday_x        bool
Temperature     float64
Fuel_Price      float64
MarkDown1       float64
MarkDown2       float64
MarkDown3       float64
MarkDown4       float64
MarkDown5       float64
CPI             float64
Unemployment    float64
IsHoliday_y        bool
Size              int64
Type_B             bool
Type_C             bool
dtype: object
   Store  Dept  IsHoliday_x  Temperature  Fuel_Price  MarkDown1  MarkDown2  \
0      1     1        False        42.31       2.572        0.0        0.0   
1      1     1         True        38.51       2.548        0.0        0.0   
2      1     1        False        39.93       2.514        0.0        0.0   
3      1     1        False        46.63       2.561        0.0        0.0   
4      1     1        False        46.50       2.625        0.0        0.0   

   MarkDown3  MarkDown4  MarkDown5         CPI  Unemployment  IsHoliday_y  \
0        0.0        0.0        0.0  211.096358         8

In [9]:
# Build the ANN model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))  # regression output

c:\Users\varsh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [11]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
8432/8432 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - loss: 535237120.0000 - mae: 14804.5596 - val_loss: 474974400.0000 - val_mae: 14598.3164
Epoch 2/50
8432/8432 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - loss: 459872192.0000 - mae: 14495.2100 - val_loss: 470590656.0000 - val_mae: 14501.7422
Epoch 3/50
8432/8432 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - loss: 458616544.0000 - mae: 14448.1250 - val_loss: 462769152.0000 - val_mae: 14131.8184
Epoch 4/50
8432/8432 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - loss: 450200832.0000 - mae: 14208.7705 - val_loss: 447548768.0000 - val_mae: 14087.9189
Epoch 5/50
8432/8432 ━━━━━━━━━━━━━━━━━━━━ 29s 3ms/step - loss: 443621056.0000 - mae: 13879.1660 - val_loss: 432489696.0000 - val_mae: 13563.6973
Epoch 6/50
8432/8432 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 421537824.0000 - mae: 13475.3545 - val_loss: 419911744.0000 - val_mae: 13381.9268
Epoch 7/50
8432/8432 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 408315680.0000 - mae: 13177.1357 - val_loss: 404436736.0000 - val_m

In [12]:
#Evaluate the Model
predictions = model.predict(X_test)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R^2 Score: {r2:.2f}")


2635/2635 ━━━━━━━━━━━━━━━━━━━━ 2s 628us/step
MAE: 11170.21
MSE: 337938925.51
RMSE: 18383.12
R^2 Score: 0.35


In [ ]:
# Save the model
model.save('../models/ann_sales_model.h5')

NameError: name 'dump' is not defined

In [15]:
import joblib

# Save the scaler
joblib.dump(scaler, '../models/scaler.joblib')


['../models/scaler.joblib']